# Part 2: MLP

In this part of the assignment, you will complete a Do-It-Yourself (DIY) implementation of a multilayer perceptron (MLP), including code to optimize the network with backpropagation and minibatch stochastic gradient descent, that corresponds to the Scikit-Learn API.

**Learning objectives.** You will:
1. Write object-oriented code for a neural network class in Python class, matching standard API patterns.
2. Apply numerical Python (NumPy) to efficiently implement a multilayer perceptron with backpropagation and minibatch stochastic gradient descent. 
3. Evaluate your implementation compared to the Scikit-Learn standard on real image data. 

The following code imports a dataset consisting of 8 by 8 pixel grayscale images of handwritten digits. Some images are visualized and then the data are flattened into 64-value one-dimensional NumPy arrays before splitting into training and testing sets.

In [1]:
# Run but DO NOT MODIFY this code

from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

digits = load_digits()
print(digits.data.shape)

# visualizing examples
_, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
for ax, image, label in zip(axes, digits.images, digits.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title("Training: %i" % label)

# flatten the images
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Split data into 70% train and 30% test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.3, shuffle=False
    )

ImportError: dlopen(/opt/anaconda3/lib/python3.12/site-packages/scipy/linalg/_fblas.cpython-312-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib
  Referenced from: <1DB3A065-D076-3961-933C-5C0F7CCA05E8> /opt/anaconda3/lib/python3.12/site-packages/scipy/linalg/_fblas.cpython-312-darwin.so
  Reason: tried: '/opt/anaconda3/lib/python3.12/site-packages/scipy/linalg/../../../../liblapack.3.dylib' (no such file), '/opt/anaconda3/lib/python3.12/site-packages/scipy/linalg/../../../../liblapack.3.dylib' (no such file), '/opt/anaconda3/bin/../lib/liblapack.3.dylib' (no such file), '/opt/anaconda3/bin/../lib/liblapack.3.dylib' (no such file), '/usr/local/lib/liblapack.3.dylib' (no such file), '/usr/lib/liblapack.3.dylib' (no such file, not in dyld cache)

## Task 1

Using Scikit-Learn, build a [multilayer perceptron classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) fit on the training data to predict the value of `y` given `X`. You may need to experiment with different numbers of hidden units (you only need to have one hidden layer) or other training hyperparameters. Ensure that you achieve a **training** [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) of at least 90%.   

Then evaluate and report the [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) of your model on the **test** data.

In [14]:
# Write code for task 1 here
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import numpy as np

mlp = MLPClassifier()
mlp.fit(X_train, y_train)
y_pred_train = mlp.predict(X_train)
y_pred_test = mlp.predict(X_test)
acc_train = accuracy_score(y_pred_train, y_train)
acc_test = accuracy_score(y_pred_test, y_test)
print(f"Train Accuracy: {acc_train:.4f}")
print(f"Test Accuracy: {acc_test:.4f}")

Train Accuracy: 1.0000
Test Accuracy: 0.9222


## Task 2

Implement a do it yourself (DIY) multilayer perceptron classifier following the Scikit-Learn API by completing the `NeuralNet` class below. 

You will see in the description of the class that we are suggesting a somewhat atypical architecture consisting of a single hidden layer with hyperbolic tangent activation followed by linear output, trained on the sum of squares loss. This is to simplify the implementation and align it with the simple example of backpropagation shown in the [Bishop Deep Learning Book](https://www.bishopbook.com/) Section 8.1.3 (meaning you do not need to derive the derivatives for backpropagation yourself and are invited to implement the derivations from the book).

Some important notes about the implementation:

1. Remember that the Scikit-Learn API treats an input `X` array, whether to `fit` or `predict`, as a matrix with a row for every data point and a column for every feature. 
2. For `fit`, every row in `X` corresponds to a given output in `y`, and  you don't need to return anything, just optimize the internal model weights using minibatch stochastic gradient descent. Model weights should be stored as instance variables -- we recommend that you initialize these weights as random normally distributed values, for example by using NumPy's [random.normal](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html). You may wish to separately implement a `backprop` helper method to compute the gradient in order to simplify the code structure, but this is optional. 
3. For `predict_proba`, you should return a two-dimensional NumPy array with a row for every row in `X` and a column for every value in `0, 1, ..., out_size - 1` corresponding to the possible classes, where each entry in a given row corresponds to a probability the given row's input corresponds to the given column's class.
4. For `predict`, you should return a one-dimensional NumPy array with a single element `0, 1, ..., out_size - 1` per row in `X` corresponding to the class with the highest predicted probability.
5. You do **not** need to include a bias term for this implementation.
6. The `fit` method takes an optional `verbose` parameter. While it is not required, we highly recommend that you include code in the `fit` method that, when `verbose` is `True`, logs/prints more information about the training process to help debug. 
7. The `pass` statements are syntactic placeholders that should be removed when you implement a method.
8. Make sure to double check the shapes/dimensions of any matrices or vectors to confirm they align with your expectations.
9. Finally, note that your implementation will be much more efficient if you use vectorized NumPy operations and avoid for loops or nested for loops over large amounts of data.

In [17]:
# Write code for task 2 here

class NeuralNet:
    """
    Basic implementation of a multilayer perceptron for multiclassification
    with a single hidden layer, hyperbolic tangent activation, and linear output. 
    Trains using a constant learning rate for a specified number of epochs using
    minibatch stochastic gradient descent with sum of squares loss. Assumes
    output classes are simply 0, 1, ..., out_size - 1.
    """
    def __init__(self, in_size=None, out_size=None, 
                 h_size=100, lr=0.01, batch_size=64,
                 epochs=2, random_state=2024):
        # todo: complete constructor/initialization
        self.h_size = h_size
        self.lr = lr
        self.batch_size = batch_size
        self.epochs = epochs
        self.random_state = random_state
        self.in_size = in_size
        self.out_size = out_size
        self.h_size = h_size
        
    def activation(self, X):
        return np.tanh(X)
    

    def predict_proba(self, X):
        """Predict probabilities for each row in X for each class in out_size"""
        pre_act_hidden = np.matmul(X, self.weights_in_hidden)
        hidden_output = self.activation(pre_act_hidden)
        output_vals = np.matmul(hidden_output, self.weights_hidden_out)
        return output_vals
    

    def predict(self, X):
        """Predict class for each row in X for each class in out_size"""
        probs = self.predict_proba(X)
        return np.argmax(probs, axis=1)
    

    def fit(self, X, y, verbose=False):
        """Train on inputs X with labels y using backprop and minibatch SGD"""
        np.random.seed(self.random_state)
        n_samples = X.shape[0]
        self.in_size = X.shape[1]
        self.out_size = len(np.unique(y))

        # Initialize weights
        self.weights_in_hidden = np.random.randn(self.in_size, self.h_size) * np.sqrt(2. / self.in_size)
        self.weights_hidden_out = np.random.randn(self.h_size, self.out_size) * np.sqrt(2. / self.in_size)

        for epoch in range(self.epochs):
            indices = np.random.permutation(n_samples)
            # Shuffle data at start of epoch
            X_shuffled, y_shuffled = X[indices], y[indices]
            for i in range(0, n_samples, self.batch_size):
                # Get batch
                X_batch = X_shuffled[i:i+self.batch_size]
                y_batch = y_shuffled[i:i+self.batch_size]
                y_batch_one_hot = np.eye(self.out_size)[y_batch]


                # Compute values at all hidden & output nodes
                hidden_vals = np.tanh(X_batch @ self.weights_in_hidden)
                out_vals = hidden_vals @ self.weights_hidden_out

                # Compute loss and delta values (from Bishop)
                loss = np.sum((out_vals - y_batch_one_hot) ** 2) / X_batch.shape[0]
                delta_out = out_vals - y_batch_one_hot
                delta_hidden = (1 - hidden_vals**2) * (delta_out @ self.weights_hidden_out.T)
                # delta_out is batch_size x out_size, weights_hidden_out is h_size x out_size

                # Compute gradients from deltas (from Bishop)
                gradient_weights_i_h = (X_batch.T @ delta_hidden)
                gradient_weights_h_o = (hidden_vals.T @ delta_out)

                # Update weights
                self.weights_in_hidden -= self.lr * gradient_weights_i_h / self.batch_size
                self.weights_hidden_out -= self.lr * gradient_weights_h_o / self.batch_size

            if verbose:
                print(f"Epoch {epoch+1} of {self.epochs}, Loss = {loss:.4f}")

## Task 3

Show that your `NeuralNet` class can effectively learn. Specifically fit an instance of your DIY model on the training data to predict the value of `y` given `X`. You may need to experiment with different numbers of hidden units (you only need to have one hidden layer) or other training hyperparameters. Ensure that you achieve a **training** [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) of at least 70% (your implementation is not expected to achieve the same level of performance as the Scikit-Learn implementation, but should demonstrate the proof of concept).   

Then evaluate and report the [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) of your model on the **test** data.

In [18]:
# Write code for task 3 here
nn = NeuralNet(epochs=50)
nn.fit(X_train, y_train)
y_pred_nn = nn.predict(X_test)
acc_nn = accuracy_score(y_pred_nn, y_test)
print(f"Test Accuracy: {acc_nn:.4f}")

Test Accuracy: 0.7870
